# MaD Conf Plots

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp

import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib widget
#%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
sns.set(style='ticks', context='talk')
#plt.rcParams['figure.figsize'] = (15,5)
plt.rcParams['figure.figsize'] = (10,5)
plt.close('all')

save_fig = True

palette = bp.colors.fau_palette
sns.set_palette(palette)
palette

In [ ]:
pg.options['round'] = 3

In [ ]:
data_path = Path("/Volumes/MaD_Lab/StudyData/HealthPsychology/CARWatch/Data")
export_path = Path("../../../exports")
result_path = Path("../../../results")

plot_path_car = result_path.joinpath("madconf_2021_02/plots/example_car")
plot_path_sleep = result_path.joinpath("madconf_2021_02/plots/example_sleep_processing")

plot_path_car.mkdir(exist_ok=True, parents=True)
plot_path_sleep.mkdir(exist_ok=True, parents=True)

## Cortisol

In [ ]:
cort_samples = pd.read_csv(export_path.joinpath("cortisol_samples_cleaned.csv"))
# assign each night an unique_id to allow repeated measures analyses and insert into dataframe
cort_samples.insert(2,'night_id', cort_samples['subject'] + '_' + cort_samples['night'].astype(str))
cort_samples = cort_samples.drop(columns='time')
cort_samples = cort_samples.set_index(list(cort_samples.columns.drop('cortisol')))

cort_samples.head()

In [ ]:
pd.DataFrame(cort_samples.xs("Unknown Alarm", level="condition").unstack().iloc[19])

In [ ]:
ylabel = "Cortisol [nmol/l]"
xlabel = "Time after Awakening [min]"

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.pointplot(data=cort_samples.xs("KS28N", level='subject').xs(0, level='night').reset_index(), x='sample', y='cortisol', ax=ax, ci=None, err_style='bars', err_kws={'capsize': 5})
ax.set_ylabel(ylabel)
ax.set_xticklabels([i*15 for i in range(5)])
ax.set_xlabel(xlabel)
ax.set_ylim([0, 25])
fig.tight_layout()

if save_fig:
    fig.savefig(plot_path_car.joinpath("img_car_example.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.pointplot(data=cort_samples.reset_index(), x='sample', y='cortisol', ax=ax, ci='sd', err_style='bars', capsize=0.2)
ax.set_ylabel(ylabel)
ax.set_xticklabels([i*15 for i in range(5)])
ax.set_xlabel(xlabel)
ax.set_ylim([0, 25])

fig.tight_layout()
if save_fig:
    fig.savefig(plot_path_car.joinpath("img_car_mean_sd.pdf"), transparent=True)

## IMU

In [ ]:
imu_path = data_path.joinpath("Sleep_Data/AK12I/NilsPodX-5DCC_20191201_234759.bin")
data, fs = bp.io.nilspod.load_dataset_nilspod(imu_path)
data = data.between_time("23:30", "07:00")

In [ ]:
sleep_results = bp.sleep.sleep_processing_pipeline.predict_pipeline_acceleration(data, fs, convert_to_g=True)
sleep_endpoints = sleep_results['sleep_endpoints']

In [ ]:
major_wear_block = bp.signals.imu.wear_detection.WearDetection.get_major_wear_block(sleep_results['wear_detection'])

In [ ]:
wear_start = major_wear_block[0]
wear_stop = major_wear_block[1]

#### Raw ACC

In [ ]:
fig, ax = plt.subplots(figsize=(14,5))
bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], downsample_factor=10*fs, ax=ax)
ax.legend().remove()
fig.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0))
fig.tight_layout(rect=(0, 0, 0.85, 1.0))

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_00_sleep_raw.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(14,5))
bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], downsample_factor=10*fs, ax=ax)

ax.axvspan(wear_start, wear_stop, color=bp.colors.fau_color('med'), alpha=0.1, label="Wear Period")

ax.legend().remove()
fig.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0))
fig.tight_layout(rect=(0, 0, 0.85, 1.0))

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_01_sleep_wear_detection.pdf"), transparent=True)

In [ ]:
data_cut = data.loc[wear_start:(wear_stop - pd.Timedelta("1:30:00"))]

In [ ]:
sleep_results = bp.sleep.sleep_endpoints.predict_pipeline(data_cut, fs)
sleep_endpoints = sleep_results['sleep_endpoints']

In [ ]:
fig, ax = plt.subplots(figsize=(14,5))
bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], sleep_endpoints=sleep_endpoints, downsample_factor=10*fs, ax=ax, plot_sleep_onset=False, plot_wake_onset=False, plot_sleep_wake=False)

ax.axvspan(wear_start, wear_stop, color=bp.colors.fau_color('med'), alpha=0.1, label="Wear Period", zorder=0)

ax.set_title(None)
ax.legend().remove()
fig.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0))
fig.tight_layout(rect=(0, 0, 0.85, 1.0))

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_02_sleep_mrp.pdf"), transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(14,5))
bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], sleep_endpoints=sleep_endpoints, downsample_factor=10*fs, ax=ax)

ax.axvspan(major_wear_block[0], pd.Timestamp(sleep_endpoints['sleep_onset']), color=bp.colors.fau_color('med'), alpha=0.1, label="Wear Period", zorder=0)
ax.axvspan(pd.Timestamp(sleep_endpoints['wake_onset']), major_wear_block[1], color=bp.colors.fau_color('med'), alpha=0.1, zorder=0)

ax.set_title(None)
ax.legend().remove()
fig.legend(loc='upper right', bbox_to_anchor=(1.0, 1.0))
fig.tight_layout(rect=(0, 0, 0.85, 1.0))

if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_03_sleep_total.pdf"), transparent=True)

## IMU – Subplots

In [ ]:
imu_path = data_path.joinpath("Sleep_Data/AK12I/NilsPodX-5DCC_20191201_234759.bin")
data, fs = bp.io.nilspod.load_dataset_nilspod(imu_path)
data = data.between_time("23:30", "06:00")

In [ ]:
sleep_results = bp.sleep.sleep_endpoints.predict_pipeline(data, fs, convert_to_g=True)
sleep_endpoints = sleep_results['sleep_endpoints']

In [ ]:
major_wear_block = bp.signals.imu.wear_detection.WearDetection.get_major_wear_block(sleep_results['wear_detection'])
wear_start = major_wear_block[0]
wear_stop = major_wear_block[1]

In [ ]:
from matplotlib.dates import DateFormatter

fig, axs = plt.subplots(figsize=(22,5), ncols=4, sharey=True)

bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], downsample_factor=10*fs, ax=axs[0])

bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], downsample_factor=10*fs, ax=axs[1])
axs[1].axvspan(wear_start, wear_stop, color=bp.colors.fau_color('med'), alpha=0.1, label="Wear Period")

#data_cut = data.loc[wear_start:(wear_stop - pd.Timedelta("00:30:00"))]
#sleep_results = bp.sleep.sleep_endpoints.predict_pipeline(data_cut, fs)
#sleep_endpoints = sleep_results['sleep_endpoints']

bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], sleep_endpoints=sleep_endpoints, downsample_factor=10*fs, ax=axs[2], plot_sleep_onset=False, plot_wake_onset=False, plot_sleep_wake=False)
axs[2].axvspan(wear_start, wear_stop, color=bp.colors.fau_color('med'), alpha=0.1, label="Wear Period", zorder=0)

bp.sleep.plotting.sleep_imu_plot(data, datastreams=['acc'], sleep_endpoints=sleep_endpoints, downsample_factor=10*fs, ax=axs[3])
axs[3].axvspan(major_wear_block[0], pd.Timestamp(sleep_endpoints['sleep_onset']), color=bp.colors.fau_color('med'), alpha=0.1, label="Wear Period", zorder=0)
axs[3].axvspan(pd.Timestamp(sleep_endpoints['wake_onset']), major_wear_block[1], color=bp.colors.fau_color('med'), alpha=0.1, zorder=0)

for ax in axs:
    ax.xaxis.set_major_formatter(DateFormatter("%H:%M"))
    h, l = ax.get_legend_handles_labels()
    ax.legend().remove()
    ax.set_title(None)
    
fig.legend(h, l, loc='upper right', bbox_to_anchor=(1.0, 1.0))
fig.tight_layout(pad=0.0, rect=(0, 0, 0.89, 1.0))
if save_fig:
    fig.savefig(plot_path_sleep.joinpath("img_sleep_pipeline_total.pdf"), transparent=True)